**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, Adam
from keras.layers import Input, Conv2D, MaxPooling2D, Activation, AveragePooling2D, Reshape
from keras.layers import BatchNormalization, Flatten

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self, s, train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function ```act``` reports for the action choice of the ```Agent```  regarding the regime where he is and according to the state $s$.

Indeed, during training, the agent acts using an $\epsilon$-greedy algorithm when he explores (sampling randomly the chosen action) with probability $\epsilon$, and tries to maximizes his reward using the ```learned_act``` function with probability $1-\epsilon$.

In test session, the ```Agent``` acts greedily and does not try to explore anymore. Therefore, he always tries to compute the best possible action regarding the state $s$.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size, grid_size))
        self.position = np.zeros((grid_size, grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train = 300 # set small when debugging
epochs_test = 100 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The arrays ```position``` and ```board``` are variables of the game caracterizing the state of our environment.
In the ```board```, we save the information about the bonuses and maluses of the environment, regarding if there is cheese or poison on the cell. The ```position``` array informs the rat of the possible moves with 0 values and where it is forbidden to go with -1 values (walls, obstacles ...).

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        """
        The Random Agent chooses his action without trying to learn
        or optimize anything. Therefore, the action choice does not depend
        on the state s and is uniformly random on the action space.
        """
        return np.random.uniform(0,self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent, env, epochs, prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

        # Save as a mp4
        if e % 10 == 0:
            env.draw(e)

        # Update stats
        score += win-lose

#         print("Epoch {e} | Loss {loss:.4f} | Win/lose count {w}/{l} ({r})"\
#               .format(e=e, loss=loss, w=win, l=lose, r=win/lose))
        agent.save()
        
        # Save as a mp4
        env.draw(prefix+str(e+1))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})".format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='./random_agent/random')

Win/lose count 1.5/1.0. Average score (1.0)
Win/lose count 3.0/7.0. Average score (-3.5)
Win/lose count 2.0/2.0. Average score (-2.3333333333333335)
Win/lose count 2.5/0. Average score (-0.5)
Win/lose count 1.5/2.0. Average score (-0.6)
Win/lose count 1.0/4.0. Average score (-1.5)
Win/lose count 1.0/1.0. Average score (-1.2857142857142858)
Win/lose count 2.0/1.0. Average score (-0.875)
Win/lose count 1.5/6.0. Average score (-1.7777777777777777)
Win/lose count 1.0/3.0. Average score (-2.0)
Win/lose count 0.5/4.0. Average score (-2.4545454545454546)
Win/lose count 1.0/1.0. Average score (-2.25)
Win/lose count 3.5/2.0. Average score (-1.8461538461538463)
Win/lose count 0.5/2.0. Average score (-1.9285714285714286)
Win/lose count 2.5/0. Average score (-1.4666666666666666)
Win/lose count 1.0/6.0. Average score (-2.0)
Win/lose count 2.0/4.0. Average score (-2.1176470588235294)
Win/lose count 3.5/1.0. Average score (-1.7222222222222223)
Win/lose count 2.0/1.0. Average score (-1.526315789473684

In [9]:
HTML(display_videos('./random_agent/random100.mp4'))

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




Let $Q^{\pi}(s,a)$ be the $Q-$function:

\begin{equation*}Q^\pi(s,a)= \mathbb{E}_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}
Thefore:

\begin{align*}
Q^\pi(s,a) & = \mathbb{E}\left[\sum_{t\geq 0} \gamma^t r(s_t, a_t) \big| s_0=s, \pi \right] \\
& = r(s,a) + \mathbb{E}\left[\sum_{t\geq 1} \gamma^t r(s_t,a_t) | s_0=s,\pi \right] \\ 
 & = r(s,a) + \gamma \sum_{\bar{s_1}} \mathbb{P}(s_1=\bar{s_1}|s_0=s;\pi(s_0))\mathbb{E}\left[\sum_{t\geq 1} \gamma^{t-1} r(s_t,a_t) \big| s_1=\bar{s_1};\pi \right] \\
&= \mathbb{E}_{(s',a')\sim p(.|s,a)}\left[r(s,a)+\gamma Q^{\pi}(s',a')\right]
\end{align*}




Then expanding the formula to the optimal Q-value function $Q^*$:

\begin{align*}
Q^*(s,a) & = max_{\pi} Q^{\pi}(s,a) \\
& = \max_{\pi} \left[ r(s, a) + \gamma \sum_{\bar{s_1}} \mathbb{P}(s_1=\bar{s_1}|s_0=s) Q^{\pi}(\bar{s_1}, \pi(s)) \right] \\ 
 & = r(s, a) + \gamma \sum_{\bar{s_1}} \mathbb{P}(s_1=\bar{s_1}|s_0=s)  \max_{\bar{a}}\left(Q^{\pi}(\bar{s_1}, \bar{a})\right) \\
 &= \mathbb{E}_{(s')\sim \pi^{*}(.|s,a)}\left[r(s,a)+\gamma \max_{a'}(Q^{\pi}(s',a')) \right] \\
\end{align*}


Finally, we want to approximate the $Q^{*}$ function. To do so, one possible criteria which can be minimized for this purpose is the quadratic difference in the last equality, that is considering the following loss function:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

where $\theta$ are parameters of our Q-value function estimation.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [8]:
from collections import deque

class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = deque(maxlen=max_memory)

    def remember(self, m):
        self.memory.append(m)

    def random_access(self):
        return self.memory[np.random.randint(0, len(self.memory))]


***
The pipeline we will use for training is given below:

In [9]:
def train(agent, env, epoch, prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [10]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16, n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        target_q = self.model.predict(s.reshape(-1, 5, 5, self.n_state)).ravel()
        return np.argmax(target_q)

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        input_next_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        
        trajectories = [self.memory.random_access() for i in range(self.batch_size)]
        for i in range(self.batch_size):
            s, next_s, _, _, _ = trajectories[i]
            
            input_states[i] = s
            input_next_states[i] = next_s
        
        q = self.model.predict(input_states)
        target_q = q.copy()
        q_next = self.model.predict(input_next_states)
        
        for i in range(self.batch_size):
            s, next_s, action, reward, _ = trajectories[i]      
            target_q[i, action] = reward + self.discount * np.max(q_next[i])
        
        # Clipping to avoid gradient exploiding
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self, name_weights='model.h5', name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        
        model.add(Flatten(input_shape=(5, 5, self.n_state)))
        
        # Input Layer of state size(n_state) and Hidden Layer with 24 nodes
        model.add(Dense(24, activation='relu'))
        
        # Hidden layer with 24 nodes
        model.add(Dense(4, activation='linear'))
        
        # Create the model based on the information above
        model.compile(loss='mse',
                      optimizer=sgd(lr=lr, decay=1.5e-4, momentum=0.6))
        
        model.summary()

        self.model = model
        

In [45]:
env = Environment(grid_size=size, max_time=T, temperature=temperature)
agent = DQN_FC(size, lr=.1, epsilon = 0.08, memory_size=2000, batch_size = 64)
train(agent, env, epochs_train, prefix='./fc_dqn_agent/fc_train')


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 50)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 24)                1224      
_________________________________________________________________
dense_8 (Dense)              (None, 4)                 100       
Total params: 1,324
Trainable params: 1,324
Non-trainable params: 0
_________________________________________________________________
Epoch 000/300 | Loss 0.0048 | Win/lose count 1.5/7.0 (-5.5)
Epoch 001/300 | Loss 0.0032 | Win/lose count 0.5/1.0 (-0.5)
Epoch 002/300 | Loss 0.0026 | Win/lose count 2.5/9.0 (-6.5)
Epoch 003/300 | Loss 0.0039 | Win/lose count 4.0/5.0 (-1.0)
Epoch 004/300 | Loss 0.0028 | Win/lose count 5.5/2.0 (3.5)
Epoch 005/300 | Loss 0.0035 | Win/lose count 3.0/2.0 (1.0)
Epoch 006/300 | Loss 0.0106 | Win/lose count 1

In [13]:
HTML(display_videos('./fc_dqn_agent/fc_train290.mp4'))

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [11]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        
        model.add(Conv2D(64, (4,4), input_shape=(5, 5, self.n_state), padding='SAME'))
        model.add(Activation('relu'))
        
        model.add(Conv2D(32, (4,4), padding='SAME'))
        model.add(Activation('relu'))
        
        model.add(Flatten())
        
        model.add(Dense(4, activation='linear'))
        
        model.compile(loss='mse',
                      optimizer=sgd(lr=lr, decay=1.5e-4, momentum=0.6))

        model.summary()
        
        self.model = model

In [48]:
env = Environment(grid_size=size, max_time=T, temperature=temperature)
agent = DQN_CNN(size, lr=.1, epsilon = 0.08, memory_size=2000, batch_size = 64)
train(agent,env,epochs_train,prefix='./cnn_dqn_agent/cnn_train')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 5, 5, 64)          2112      
_________________________________________________________________
activation_1 (Activation)    (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 5, 5, 32)          32800     
_________________________________________________________________
activation_2 (Activation)    (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 4)                 3204      
Total params: 38,116
Trainable params: 38,116
Non-trainable params: 0
_________________________________________________________________
Epoch 

In [16]:
HTML(display_videos('./cnn_dqn_agent/cnn_train270.mp4'))

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [76]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.08, memory_size=2000, batch_size = 64)
agent_cnn.load(name_weights='./cnn_dqn_agent/cnn_trainmodel.h5',name_model='./cnn_dqn_agent/cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.08, memory_size=2000, batch_size = 64)
agent_fc.load(name_weights='./fc_dqn_agent/fc_trainmodel.h5',name_model='./fc_dqn_agent/fc_trainmodel.json')

print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='./cnn_dqn_agent/cnn_test')

env.reset()
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='./fc_dqn_agent/fc_test')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 5, 5, 64)          2112      
_________________________________________________________________
activation_15 (Activation)   (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 5, 5, 32)          32800     
_________________________________________________________________
activation_16 (Activation)   (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_17 (Flatten)         (None, 800)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 4)                 3204      
Total params: 38,116
Trainable params: 38,116
Non-trainable params: 0
_________________________________________________________________
______

In [20]:
HTML(display_videos('./cnn_dqn_agent/cnn_test96.mp4'))

In [21]:
HTML(display_videos('fc_dqn_agent/fc_test98.mp4'))

## Comments

After performing the testing session on the FC DQN and CNN DQN, we notice the following properties:
- Both agents learn how to maximize their profit (reward) through episodes and achieve great scores.
- The CNN version outperforms in average the FC DQN agent.
- Increasing the temperature implies having fewer poisons in our environment, and much more bonuses (cheese). Therefore, the overall score increases when temperature goes up.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [12]:
def train_explore(agent, env, epoch, prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state, train=True)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size, grid_size))
        self.position = np.zeros((grid_size, grid_size))
        self.malus_position = np.zeros((grid_size, grid_size,1))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b

    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        
        if train:
            reward -= self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1
        reward = reward + self.board[self.x, self.y]
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over
    
    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[:2,:]= -1
        self.position[:, :2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        
        self.board[self.x,self.y] = 0
        self.board[self.position < 0] = 0
        
        self.t = 0
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [24]:
# Training
temperature = 0.3
epochs_train = 201
epochs_test = 50
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=temperature)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 64, n_state=3)
train_explore(agent, env, epochs_train, prefix='./cnn_explore_dqn_agent/cnn_train_explore')


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 5, 5, 64)          3136      
_________________________________________________________________
activation_3 (Activation)    (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 5, 5, 32)          32800     
_________________________________________________________________
activation_4 (Activation)    (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 3204      
Total params: 39,140
Trainable params: 39,140
Non-trainable params: 0
_________________________________________________________________
Epoch 

In [25]:
HTML(display_videos('./cnn_explore_dqn_agent/cnn_train_explore190.mp4'))

In [107]:
# Evaluation
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=temperature)
test(agent,env,epochs_test,prefix='./cnn_explore_dqn_agent/cnn_test_explore')
HTML(display_videos('./cnn_explore_dqn_agent/cnn_test_explore50.mp4'))

Win/lose count 50.0/14.0. Average score (72.0)
Win/lose count 33.0/2.0. Average score (67.0)
Win/lose count 38.0/6.0. Average score (66.0)
Win/lose count 40.0/6.0. Average score (66.5)
Win/lose count 49.0/6.0. Average score (70.4)
Win/lose count 54.0/6.0. Average score (74.66666666666667)
Win/lose count 32.0/0. Average score (73.14285714285714)
Win/lose count 44.0/6.0. Average score (73.5)
Win/lose count 31.0/0. Average score (72.22222222222223)
Win/lose count 27.0/4.0. Average score (69.6)
Win/lose count 42.0/8.0. Average score (69.45454545454545)
Win/lose count 36.0/12.0. Average score (67.66666666666667)
Win/lose count 23.0/8.0. Average score (64.76923076923077)
Win/lose count 17.0/4.0. Average score (62.0)
Win/lose count 40.0/6.0. Average score (62.4)
Win/lose count 39.0/6.0. Average score (62.625)
Win/lose count 22.0/10.0. Average score (60.35294117647059)
Win/lose count 40.0/4.0. Average score (61.0)
Win/lose count 26.0/8.0. Average score (59.68421052631579)
Win/lose count 45.0/1

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

At first, since we did not understand how to build a model that mimicks the behavior of our expert DQN (should it be an heuristic ?), we used the second idea of incorporating a decreasing $\epsilon$-greedy exploration. Therefore, we compare the two _expert_ DQN agents where we enrich the state (above) and therefore the input dimension in our deep neural networks and the model (below) where we have a decreasing exploration rate.

In [13]:
def train_explore_decreasing(agent, env, epoch, prefix=''):
    # Number of won games
    score = 0
    loss = 0
    initial_epsilon = 1.0
    final_epsilon = 1e-2
    agent.set_epsilon(initial_epsilon)
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state, train=True)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
        
        if agent.epsilon > final_epsilon:
            agent.set_epsilon(agent.epsilon - (initial_epsilon-final_epsilon)/epoch)
        
        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

In [29]:
# Training
temperature = 0.3
epochs_train = 201
epochs_test = 50
env = Environment(grid_size=size, max_time=T, temperature=temperature)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 64, n_state=2)
train_explore_decreasing(agent, env, epochs_train, prefix='./cnn_explore_decreasing_dqn_agent/cnn_train_explore')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 5, 5, 64)          2112      
_________________________________________________________________
activation_5 (Activation)    (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 5, 5, 32)          32800     
_________________________________________________________________
activation_6 (Activation)    (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 3204      
Total params: 38,116
Trainable params: 38,116
Non-trainable params: 0
_________________________________________________________________
Epoch 

In [30]:
HTML(display_videos('./cnn_explore_decreasing_dqn_agent/cnn_train_explore190.mp4'))

In [31]:
# Evaluation
agent.set_epsilon(1e-2)
test(agent,env,epochs_test,prefix='./cnn_explore_decreasing_dqn_agent/cnn_test_explore')

Win/lose count 4.5/0. Average score (9.0)
Win/lose count 15.5/2.0. Average score (18.0)
Win/lose count 16.5/0. Average score (23.0)
Win/lose count 2.5/0. Average score (18.5)
Win/lose count 14.5/0. Average score (20.6)
Win/lose count 6.5/0. Average score (19.333333333333332)
Win/lose count 7.0/0. Average score (18.571428571428573)
Win/lose count 9.5/0. Average score (18.625)
Win/lose count 5.0/0. Average score (17.666666666666668)
Win/lose count 4.5/0. Average score (16.8)
Win/lose count 23.0/2.0. Average score (19.09090909090909)
Win/lose count 9.5/2.0. Average score (18.75)
Win/lose count 3.0/1.0. Average score (17.615384615384617)
Win/lose count 9.0/0. Average score (17.642857142857142)
Win/lose count 2.0/0. Average score (16.733333333333334)
Win/lose count 7.0/1.0. Average score (16.4375)
Win/lose count 5.5/0. Average score (16.11764705882353)
Win/lose count 11.5/0. Average score (16.5)
Win/lose count 8.5/0. Average score (16.526315789473685)
Win/lose count 16.0/1.0. Average score 

In [32]:
HTML(display_videos('./cnn_explore_decreasing_dqn_agent/cnn_test_explore50.mp4'))

### Bonus question : A model that mimicks the expert DQN

In [14]:
class MimicDQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(MimicDQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape([1,s.shape[0],s.shape[1],s.shape[2]]))[0,:])

    def reinforce(self, s_, a_):
        # Two steps: first memorize the states, second learn from the pool
        self.memory.remember([s_[:,:,1:], a_]) #now we will not use the fact that our model need to verify the bellman equation but directly copy what the expert has done
        #hence we juste need the current state and the action that has been applied
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s_, a_ = self.memory.random_access()
            input_states[i] = s_
            target_q[i,a_] = 10  #Now we want to mimick the expert hence we want our target function to be maximized 
            #in the action applied by the expert
            
        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model
            
class MimicDQN_CNN(MimicDQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(MimicDQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(50,(2,2),input_shape=(5,5,self.n_state,),activation='relu')) #model with 1 hidden CNN performs the best
        #with my method from what I ve experimented
        model.add(Flatten())
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [15]:
def train_mimick(expert, agent, env, epoch, prefix=''):
    # Number of won games
    score = 0
    loss = 0    
    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        while not game_over:
            # The agent performs an action
            action = expert.act(state, train=False)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state,  action)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

In [18]:
epochs_train = 101
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=temperature)
expert = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 64, n_state=3)
train_explore(expert, env, epochs_train, prefix='./tmp')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 5, 5, 64)          3136      
_________________________________________________________________
activation_3 (Activation)    (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 5, 5, 32)          32800     
_________________________________________________________________
activation_4 (Activation)    (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 3204      
Total params: 39,140
Trainable params: 39,140
Non-trainable params: 0
_________________________________________________________________
Epoch 

In [21]:
agent_mimic = MimicDQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 64, n_state = 2)
train_mimick(expert, agent_mimic, env, 101, prefix='./cnn_mimic_dqn/mimic')

Epoch 000/101 | Loss 0.0521 | Win/lose count 2.0/0 (2.0)
Epoch 001/101 | Loss 1.5926 | Win/lose count 25.0/2.0 (23.0)
Epoch 002/101 | Loss 0.6346 | Win/lose count 17.0/0 (17.0)
Epoch 003/101 | Loss 2.8725 | Win/lose count 41.0/0 (41.0)
Epoch 004/101 | Loss 2.7814 | Win/lose count 40.0/0 (40.0)
Epoch 005/101 | Loss 1.7593 | Win/lose count 16.0/0 (16.0)
Epoch 006/101 | Loss 1.3762 | Win/lose count 6.0/0 (6.0)
Epoch 007/101 | Loss 0.6719 | Win/lose count 41.0/0 (41.0)
Epoch 008/101 | Loss 1.3592 | Win/lose count 0/0 (0)
Epoch 009/101 | Loss 2.8556 | Win/lose count 8.0/0 (8.0)
Epoch 010/101 | Loss 3.4326 | Win/lose count 43.0/0 (43.0)
Epoch 011/101 | Loss 2.2501 | Win/lose count 11.0/0 (11.0)
Epoch 012/101 | Loss 1.6072 | Win/lose count 40.0/0 (40.0)
Epoch 013/101 | Loss 1.8256 | Win/lose count 32.0/0 (32.0)
Epoch 014/101 | Loss 1.9962 | Win/lose count 32.0/0 (32.0)
Epoch 015/101 | Loss 1.7667 | Win/lose count 42.0/0 (42.0)
Epoch 016/101 | Loss 1.1371 | Win/lose count 13.0/0 (13.0)
Epoch 0

In [23]:
HTML(display_videos('./cnn_mimic_dqn/mimic90.mp4'))

In [25]:
env_test = Environment(grid_size=size, max_time=T, temperature=0.3)
test(agent_mimic, env_test, epochs_test,prefix='./cnn_mimic_dqn/test')

Win/lose count 5.0/3.0. Average score (4.0)
Win/lose count 9.5/2.0. Average score (9.5)
Win/lose count 11.0/4.0. Average score (11.0)
Win/lose count 12.5/4.0. Average score (12.5)
Win/lose count 5.0/3.0. Average score (10.8)
Win/lose count 5.5/2.0. Average score (10.166666666666666)
Win/lose count 10.0/1.0. Average score (11.285714285714286)
Win/lose count 19.5/7.0. Average score (13.0)
Win/lose count 14.5/10.0. Average score (12.555555555555555)
Win/lose count 9.0/4.0. Average score (12.3)
Win/lose count 17.5/7.0. Average score (13.090909090909092)
Win/lose count 19.0/8.0. Average score (13.833333333333334)
Win/lose count 9.0/6.0. Average score (13.23076923076923)
Win/lose count 5.0/8.0. Average score (11.857142857142858)
Win/lose count 4.0/5.0. Average score (10.933333333333334)
Win/lose count 10.0/3.0. Average score (11.125)
Win/lose count 11.0/1.0. Average score (11.647058823529411)
Win/lose count 16.0/5.0. Average score (12.222222222222221)
Win/lose count 17.5/4.0. Average score (

In [26]:
HTML(display_videos('./cnn_mimic_dqn/test100.mp4'))